In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
import matplotlib.dates as mdates

In [26]:
#import data
harvest_perrault = pd.read_csv('harvester_perrault.csv')
harvest_pilot = pd.read_csv('harvest_pilot.csv')

how many Perrault Harvester's Operating simultaneously?

In [27]:
harvest_perrault['vpm'] = harvest_perrault['vpm'] * 3 #number of perrault harvesters running per shift

Calculate Productive Time

In [28]:
harvest_perrault['productive_minutes'] = harvest_perrault['strings'] / harvest_perrault['vpm']

In [29]:
harvest_perrault['productive_minutes'].sum()

28872.851147312547

In [30]:
perrault = harvest_perrault.groupby('variety')['productive_minutes'].sum().reset_index()

In [31]:
perrault['productive_minutes'].sum()

28872.85114731255

In [32]:
perrault['variety'].unique()

array(['Citra', 'Experimental', 'HBC 1019', 'HBC 522', 'HBC 586',
       'HBC 638', 'HBC 682', 'Idaho 7', 'Mosaic', 'Palisade', 'Sabro',
       'Simcoe', 'Talus'], dtype=object)

In [33]:
# Define the order in which varieties need to be picked
picking_order = [
    'Simcoe', 'Sabro', 'Citra', 'Mosaic', 'HBC 1019', 'Palisade', 'HBC 522', 'Idaho 7', 'Talus', 'HBC 682',
    'HBC 638', 'HBC 586', 'Experimental'
]

# Sort the DataFrame based on the picking order
perrault['order'] = perrault['variety'].apply(lambda x: picking_order.index(x))
perrault = perrault.sort_values(by='order')#.drop(columns='order')


In [59]:
from datetime import datetime, timedelta
import pandas as pd

# Function to check if the given date is a working day (Monday to Saturday)
def is_working_day(date):
    return date.weekday() < 6

# Function to find the next working day starting from current_datetime
def next_working_day(current_datetime):
    next_day = current_datetime + timedelta(days=1)
    while not is_working_day(next_day):
        next_day += timedelta(days=1)
    return next_day

# Function to calculate effective shift time and break time in a shift
def calculate_shift_time_with_breaks(variety, variety_shift_lengths, work_period=240, break_duration=10, lunch_duration=30):
    shift_length = variety_shift_lengths.get(variety, 600)
    num_breaks_per_shift = shift_length // work_period
    total_break_time_per_shift = num_breaks_per_shift * break_duration
    effective_shift_length = shift_length - total_break_time_per_shift - lunch_duration
    return effective_shift_length, total_break_time_per_shift

def create_shift_schedule_with_breaks(data, start_datetime, variety_shift_lengths, daily_start_time='06:00', second_shift_start_time='17:00', break_duration=10, lunch_duration=30):
    schedule = []
    current_datetime = start_datetime

    for index, row in data.iterrows():
        variety = row['variety']
        total_time = row['productive_minutes']
        shift_length = variety_shift_lengths.get(variety, 600)  # Default to 600 if variety not found
        effective_shift_length, total_break_time_per_shift = calculate_shift_time_with_breaks(
            total_time, shift_length=shift_length
        )

        while total_time > 0:
            # Skip non-working days (Sundays)
            if not is_working_day(current_datetime):
             current_datetime = next_working_day(current_datetime)

            if current_datetime.time() < datetime.strptime(second_shift_start_time, '%H:%M').time():
                shift_start_time = datetime.combine(current_datetime.date(), datetime.strptime(daily_start_time, '%H:%M').time())
            else:
                shift_start_time = datetime.combine(current_datetime.date(), datetime.strptime(second_shift_start_time, '%H:%M').time())
                if current_datetime.time() < datetime.strptime(second_shift_start_time, '%H:%M').time():
                    shift_start_time -= timedelta(days=1)

            shift_end_time = shift_start_time + timedelta(minutes=effective_shift_length + total_break_time_per_shift)

            if shift_end_time.time() >= datetime.strptime('03:00', '%H:%M').time() and shift_start_time.time() >= datetime.strptime(second_shift_start_time, '%H:%M').time():
                shift_end_time = datetime.combine(next_working_day(shift_start_time), datetime.min.time()) + timedelta(minutes=effective_shift_length + total_break_time_per_shift)

            schedule.append({
                'variety': variety,
                'ShiftStart': shift_start_time,
                'ShiftEnd': shift_end_time,
                'ShiftTime': effective_shift_length,
                'BreakTime': total_break_time_per_shift,
                'LunchBreak': lunch_duration if effective_shift_length >= 300 else 0
            })

            total_time -= effective_shift_length
            current_datetime = shift_end_time

            if current_datetime.time() >= datetime.strptime('03:00', '%H:%M').time():
                current_datetime = next_working_day(current_datetime)
                current_datetime = datetime.combine(current_datetime, datetime.min.time()) + timedelta(hours=int(daily_start_time.split(':')[0]), minutes=int(daily_start_time.split(':')[1]))

            if current_datetime.time() >= datetime.strptime(second_shift_start_time, '%H:%M').time():
                current_datetime = datetime.combine(current_datetime.date(), datetime.strptime(second_shift_start_time, '%H:%M').time())
            else:
                current_datetime = datetime.combine(current_datetime.date(), datetime.strptime(daily_start_time, '%H:%M').time())

    return pd.DataFrame(schedule)

# Example usage
start_datetime = datetime(2024, 9, 1, 6, 0)
second_shift_start_time = '18:00'  # Define second_shift_start_time here

variety_shift_lengths = {
    'Simcoe': 600,  # 10 hours
    'Citra': 600,   # 10 hours
    'Sabro': 600,   # 10 hours
    'Mosaic': 600,  # 10 hours
    'HBC 1019': 450,  # 7.5 hours
    'Palisade': 450,  # 7.5 hours
    'HBC 522': 450,   # 7.5 hours
    'Talus': 450,     # 7.5 hours
    'HBC 682': 450,   # 7.5 hours
    'HBC 638': 450,   # 7.5 hours
    'HBC 586': 450,   # 7.5 hours
    'Experimental': 450  # 7.5 hours
}

# Assuming 'perrault' is your dataframe
shift_schedule = create_shift_schedule_with_breaks(
    data=perrault,
    start_datetime=start_datetime,
    variety_shift_lengths=variety_shift_lengths,
    daily_start_time='06:00',
    second_shift_start_time=second_shift_start_time  # 6:00 PM
)

# Add a new column with just the date in m-d-yyyy format
shift_schedule['Date'] = shift_schedule['ShiftStart'].dt.strftime('%m-%d-%Y')

# Add a week column to determine the pay period
shift_schedule['Week'] = shift_schedule['ShiftStart'].dt.to_period('W-SAT')

# Calculate Shift Hours
shift_schedule['ShiftHours'] = shift_schedule['ShiftTime'] / 60  # Convert minutes to hours
shift_schedule['ProductiveHours'] = shift_schedule['ProductiveTime'] / 60  # Convert minutes to hours

weekly_hours = shift_schedule.groupby('Week')['ShiftHours'].sum().reset_index()

# Calculate the number of shifts per week
weekly_hours['NumShifts'] = (weekly_hours['ShiftHours'] / 40).apply(lambda x: max(2, round(x)))

# Calculate Overtime Threshold
weekly_hours['OvertimeThreshold'] = weekly_hours['NumShifts'] * 40

# Calculate Overtime Hours
weekly_hours['OvertimeHours'] = weekly_hours.apply(lambda row: max(0, row['ShiftHours'] - row['OvertimeThreshold']), axis=1)

# Combine ShiftStart and ShiftEnd into a single column for easier reading
shift_schedule['Shift_Combined'] = shift_schedule['ShiftStart'].astype(str) + " - " + shift_schedule['ShiftEnd'].astype(str)

# Merge Overtime Hours back into the shift schedule
shift_schedule = shift_schedule.merge(weekly_hours[['Week', 'OvertimeHours']], on='Week', how='left')

# Function to assign shift categories (Shift A or Shift B)
def assign_shift(row):
    shift_time = row['ShiftStart'].time()
    if shift_time >= pd.to_datetime('06:00', format='%H:%M').time() and shift_time < pd.to_datetime(second_shift_start_time, format='%H:%M').time():
        return 'Shift_A'
    else:
        return 'Shift_B'

# Apply the function to create the new column
shift_schedule['ShiftCategory'] = shift_schedule.apply(assign_shift, axis=1)

# Calculate total overtime
total_overtime = weekly_hours['OvertimeHours'].sum()

# Display the final shift schedule
print(shift_schedule)








TypeError: calculate_shift_time_with_breaks() got an unexpected keyword argument 'shift_length'

In [49]:
 shift_schedule

,variety,ShiftStart,ShiftEnd,ShiftTime,BreakTime,LunchBreak,ProductiveTime,Date,Week,ShiftHours,ProductiveHours,Shift_Combined,OvertimeHours,ShiftCategory
0,Simcoe,2024-09-01 06:00:00,2024-09-01 16:00:00,550.0,20,30,550.0,09-01-2024,2024-09-01/2024-09-07,9.166667,9.166667,2024-09-01 06:00:00 - 2024-09-01 16:00:00,8.333333,Shift_A
1,Simcoe,2024-09-01 18:00:00,2024-09-02 04:00:00,550.0,20,30,550.0,09-01-2024,2024-09-01/2024-09-07,9.166667,9.166667,2024-09-01 18:00:00 - 2024-09-02 04:00:00,8.333333,Shift_B
2,Simcoe,2024-09-02 06:00:00,2024-09-02 16:00:00,550.0,20,30,550.0,09-02-2024,2024-09-01/2024-09-07,9.166667,9.166667,2024-09-02 06:00:00 - 2024-09-02 16:00:00,8.333333,Shift_A
3,Simcoe,2024-09-02 18:00:00,2024-09-03 04:00:00,550.0,20,30,550.0,09-02-2024,2024-09-01/2024-09-07,9.166667,9.166667,2024-09-02 18:00:00 - 2024-09-03 04:00:00,8.333333,Shift_B
4,Simcoe,2024-09-03 06:00:00,2024-09-03 16:00:00,550.0,20,30,550.0,09-03-2024,2024-09-01/2024-09-07,9.166667,9.166667,2024-09-03 06:00:00 - 2024-09-03 16:00:00,8.333333,Shift_A
5,Simcoe,2024-09-03 18:00:00,2024-09-04 04:00:00,550.0,20,30,550.0,09-03-2024,2024-09-01/2024-09-07,9.166667,9.166667,2024-09-03 18:00:00 - 2024-09-04 04:00:00,8.333333,Shift_B
6,Simcoe,2024-09-04 06:00:00,2024-09-04 16:00:00,550.0,20,30,550.0,09-04-2024,2024-09-01/2024-09-07,9.166667,9.166667,2024-09-04 06:00:00 - 2024-09-04 16:00:00,8.333333,Shift_A
7,Simcoe,2024-09-04 18:00:00,2024-09-05 04:00:00,550.0,20,30,550.0,09-04-2024,2024-09-01/2024-09-07,9.166667,9.166667,2024-09-04 18:00:00 - 2024-09-05 04:00:00,8.333333,Shift_B
8,Simcoe,2024-09-05 06:00:00,2024-09-05 16:00:00,550.0,20,30,550.0,09-05-2024,2024-09-01/2024-09-07,9.166667,9.166667,2024-09-05 06:00:00 - 2024-09-05 16:00:00,8.333333,Shift_A
9,Simcoe,2024-09-05 18:00:00,2024-09-06 04:00:00,550.0,20,30,550.0,09-05-2024,2024-09-01/2024-09-07,9.166667,9.166667,2024-09-05 18:00:00 - 2024-09-06 04:00:00,8.333333,Shift_B


In [24]:
simple = shift_schedule[['variety', 'ShiftCategory', 'Date', 'ShiftStart', 'ShiftEnd', 'ShiftTime']]

In [25]:
simple 

,variety,ShiftCategory,Date,ShiftStart,ShiftEnd,ShiftTime
0,Simcoe,Shift_A,09-02-2024,2024-09-02 06:00:00,2024-09-02 16:00:00,570.0
1,Simcoe,Shift_B,09-02-2024,2024-09-02 18:00:00,2024-09-03 04:00:00,570.0
2,Simcoe,Shift_A,09-03-2024,2024-09-03 06:00:00,2024-09-03 16:00:00,570.0
3,Simcoe,Shift_B,09-03-2024,2024-09-03 18:00:00,2024-09-04 04:00:00,570.0
4,Simcoe,Shift_A,09-04-2024,2024-09-04 06:00:00,2024-09-04 16:00:00,570.0
...,...,...,...,...,...,...
170,HBC 586,Shift_A,12-07-2024,2024-12-07 06:00:00,2024-12-07 13:30:00,420.0
171,HBC 586,Shift_B,12-07-2024,2024-12-07 18:00:00,2024-12-08 01:30:00,420.0
172,Experimental,Shift_A,12-09-2024,2024-12-09 06:00:00,2024-12-09 13:30:00,420.0
173,Experimental,Shift_B,12-09-2024,2024-12-09 18:00:00,2024-12-10 01:30:00,420.0


In [162]:
schedule = shift_schedule.groupby(['Week', 'ShiftCategory']) [
    ['total_hours', 'LunchBreak', 'BreakTime', 'ProcessingTime']].sum()


In [163]:
schedule 

total_hours  LunchBreak  BreakTime  \
Week                  ShiftCategory                                       
2024-09-01/2024-09-07 Shift_A               60.0         180        120   
                      Shift_B               60.0         180        120   
2024-09-08/2024-09-14 Shift_A               60.0         180        120   
                      Shift_B               60.0         180        120   
2024-09-15/2024-09-21 Shift_A               60.0         180        120   
                      Shift_B               60.0         180        120   
2024-09-22/2024-09-28 Shift_A               60.0         240         80   
                      Shift_B               45.0         180         60   
2024-09-29/2024-10-05 Shift_A               45.0         180         60   
                      Shift_B               45.0         180         60   
2024-10-06/2024-10-12 Shift_A                7.5          30         10   

                                     ProcessingTime  
Week                  ShiftCategory                  
2024-09-01/2024-09-07 Shift_A                3300.0  
                      Shift_B                3300.0  
2024-09-08/2024-09-14 Shift_A                3300.0  
                      Shift_B                3300.0  
2024-09-15/2024-09-21 Shift_A                3300.0  
                      Shift_B                3300.0  
2024-09-22/2024-09-28 Shift_A                3280.0  
                      Shift_B                2460.0  
2024-09-29/2024-10-05 Shift_A                2460.0  
                      Shift_B                2460.0  
2024-10-06/2024-10-12 Shift_A                 410.0

In [47]:
def create_variety_minutes(shift_schedule, variety_column ='variety'):

    variety_dataframes = {}
    total_mins = {}

    unique_varieties = shift_schedule[variety_column].unique()

    for variety in unique_varieties:
        variety_df = shift_schedule[shift_schedule[variety_column] == variety].copy()
        variety_dataframes[variety] = variety_df
        total_mins[variety] = variety_df['ProductiveTime'].sum()

    return variety_dataframes, total_mins

# Example usage:
variety_dfs, total_mins = create_variety_minutes(shift_schedule)

# Print total hours for each variety
for variety, total_mins in total_mins.items():
    print(f"Total hours for {variety}: {total_mins}")



Total hours for Simcoe: 9120.0
Total hours for Sabro: 1140.0
Total hours for Citra: 3420.0
Total hours for Mosaic: 6270.0
Total hours for HBC 1019: 420.0
Total hours for Palisade: 990.0
Total hours for HBC 522: 990.0
Total hours for Idaho 7: 1140.0
Total hours for Talus: 990.0
Total hours for HBC 682: 4410.0
Total hours for HBC 638: 990.0
Total hours for HBC 586: 2130.0
Total hours for Experimental: 420.0


In [37]:
def create_variety_minutes(shift_schedule, variety_column ='variety'):

    variety_dataframes = {}
    total_mins = {}

    unique_varieties = shift_schedule[variety_column].unique()

    for variety in unique_varieties:
        variety_df = shift_schedule[shift_schedule[variety_column] == variety].copy()
        variety_dataframes[variety] = variety_df
        total_mins[variety] = variety_df['BreakTime'].sum()

    return variety_dataframes, total_mins

# Example usage:
variety_dfs, total_mins = create_variety_minutes(shift_schedule)

# Print total hours for each variety
for variety, total_mins in total_mins.items():
    print(f"Total hours for {variety}: {total_mins}")

Total hours for Simcoe: 320
Total hours for Sabro: 40
Total hours for Citra: 120
Total hours for Mosaic: 220
Total hours for HBC 1019: 10
Total hours for Palisade: 20
Total hours for HBC 522: 20
Total hours for Idaho 7: 40
Total hours for Talus: 20
Total hours for HBC 682: 80
Total hours for HBC 638: 20
Total hours for HBC 586: 40
Total hours for Experimental: 10


In [38]:
perrault  

,variety,productive_minutes,order
11,Simcoe,8733.401646,0
10,Sabro,909.895632,1
0,Citra,3373.121200,2
8,Mosaic,5935.206132,3
2,HBC 1019,174.718073,4
9,Palisade,662.998767,5
3,HBC 522,910.397805,6
7,Idaho 7,830.955326,7
12,Talus,965.229734,8
6,HBC 682,3859.492678,9


In [39]:
8733.40 + 320

9053.4

In [40]:
9120-8733.40  

386.60000000000036

In [41]:
Simcoe = shift_schedule[shift_schedule['variety']== 'Simcoe']

In [42]:
Simcoe

,variety,ShiftStart,ShiftEnd,ShiftTime,BreakTime,LunchBreak,ProductiveTime,ProcessingTime,Date,Week,ShiftHours,ProductiveHours,ProcessingHours,Shift_Combined,OvertimeHours,ShiftCategory
0,Simcoe,2024-09-02 06:00:00,2024-09-02 16:00:00,570.0,20,30,570.0,550.0,09-02-2024,2024-09-01/2024-09-07,9.5,9.5,9.166667,2024-09-02 06:00:00 - 2024-09-02 16:00:00,0.0,Shift_A
1,Simcoe,2024-09-02 18:00:00,2024-09-03 04:00:00,570.0,20,30,570.0,550.0,09-02-2024,2024-09-01/2024-09-07,9.5,9.5,9.166667,2024-09-02 18:00:00 - 2024-09-03 04:00:00,0.0,Shift_B
2,Simcoe,2024-09-03 06:00:00,2024-09-03 16:00:00,570.0,20,30,570.0,550.0,09-03-2024,2024-09-01/2024-09-07,9.5,9.5,9.166667,2024-09-03 06:00:00 - 2024-09-03 16:00:00,0.0,Shift_A
3,Simcoe,2024-09-03 18:00:00,2024-09-04 04:00:00,570.0,20,30,570.0,550.0,09-03-2024,2024-09-01/2024-09-07,9.5,9.5,9.166667,2024-09-03 18:00:00 - 2024-09-04 04:00:00,0.0,Shift_B
4,Simcoe,2024-09-04 06:00:00,2024-09-04 16:00:00,570.0,20,30,570.0,550.0,09-04-2024,2024-09-01/2024-09-07,9.5,9.5,9.166667,2024-09-04 06:00:00 - 2024-09-04 16:00:00,0.0,Shift_A
5,Simcoe,2024-09-04 18:00:00,2024-09-05 04:00:00,570.0,20,30,570.0,550.0,09-04-2024,2024-09-01/2024-09-07,9.5,9.5,9.166667,2024-09-04 18:00:00 - 2024-09-05 04:00:00,0.0,Shift_B
6,Simcoe,2024-09-05 06:00:00,2024-09-05 16:00:00,570.0,20,30,570.0,550.0,09-05-2024,2024-09-01/2024-09-07,9.5,9.5,9.166667,2024-09-05 06:00:00 - 2024-09-05 16:00:00,0.0,Shift_A
7,Simcoe,2024-09-05 18:00:00,2024-09-06 04:00:00,570.0,20,30,570.0,550.0,09-05-2024,2024-09-01/2024-09-07,9.5,9.5,9.166667,2024-09-05 18:00:00 - 2024-09-06 04:00:00,0.0,Shift_B
8,Simcoe,2024-09-06 06:00:00,2024-09-06 16:00:00,570.0,20,30,570.0,550.0,09-06-2024,2024-09-01/2024-09-07,9.5,9.5,9.166667,2024-09-06 06:00:00 - 2024-09-06 16:00:00,0.0,Shift_A
9,Simcoe,2024-09-06 18:00:00,2024-09-07 04:00:00,570.0,20,30,570.0,550.0,09-06-2024,2024-09-01/2024-09-07,9.5,9.5,9.166667,2024-09-06 18:00:00 - 2024-09-07 04:00:00,0.0,Shift_B


In [43]:
Simcoe['ProcessingTime'].sum()

8800.0

In [44]:
Simcoe['BreakTime'].sum()

320

In [45]:
8800+320

9120

In [74]:
from datetime import datetime, timedelta
import pandas as pd

# Function to check if the given date is a working day (Monday to Saturday)
def is_working_day(work_day):
    # Monday to Saturday are working days (0 = Monday, 6 = Sunday)
    return work_day.weekday() < 6

# Function to find the next working day starting from current_datetime
def next_working_day(current_datetime):
    next_day = current_datetime + timedelta(days=1)
    while not is_working_day(next_day):
        next_day += timedelta(days=1)
    return next_day

# Function to calculate effective shift time, break time, and processing time in a shift
def calculate_shift_time_with_breaks(work_period=240, break_duration=10, lunch_duration=30, shift_length=600):
    num_breaks_per_shift = shift_length // work_period
    total_break_time_per_shift = num_breaks_per_shift * break_duration
    effective_shift_length = shift_length - total_break_time_per_shift - lunch_duration
    productive_time_including_breaks = shift_length - lunch_duration
    processing_time = effective_shift_length
    return effective_shift_length, total_break_time_per_shift, productive_time_including_breaks, processing_time

# Function to create shift schedule with breaks
def create_shift_schedule_with_breaks(data, start_datetime, variety_shift_lengths, daily_start_time='06:00', second_shift_start_time='18:00', break_duration=10, lunch_duration=30):
    schedule = []
    current_datetime = start_datetime

    for index, row in data.iterrows():
        variety = row['variety']
        total_time = row['productive_minutes']
        shift_length = variety_shift_lengths.get(variety, 600)
        effective_shift_length, total_break_time_per_shift, productive_time_including_breaks, processing_time = calculate_shift_time_with_breaks(shift_length=shift_length)

        while total_time > 0:
            # Skip non-working days (Sundays)
            if not is_working_day(current_datetime):
                current_datetime = next_working_day(current_datetime)

            # First Shift (06:00 - 16:00)
            if current_datetime.time() < datetime.strptime('16:00', '%H:%M').time():
                shift_start_time = datetime.combine(current_datetime.date(), datetime.strptime(daily_start_time, '%H:%M').time())
                shift_end_time = datetime.combine(shift_start_time.date(), datetime.strptime('16:00', '%H:%M').time())

                schedule.append({
                    'variety': variety,
                    'ShiftStart': shift_start_time,
                    'ShiftEnd': shift_end_time,
                    'ShiftTime': productive_time_including_breaks,  # Productive time includes breaks
                    'BreakTime': total_break_time_per_shift,
                    'LunchBreak': lunch_duration,
                    'ProductiveTime': productive_time_including_breaks,  # Productive Time includes breaks but excludes lunch
                    'ProcessingTime': processing_time  # Processing time excludes both breaks and lunch
                })
                total_time -= productive_time_including_breaks
                current_datetime = shift_end_time

            # Second Shift (18:00 - 04:00)
            if total_time > 0:
                # Check if it's Saturday evening and ensure the shift starts at 18:00
                if current_datetime.weekday() == 5 and current_datetime.time() < datetime.strptime('18:00', '%H:%M').time():
                    current_datetime = datetime.combine(current_datetime.date(), datetime.strptime(second_shift_start_time, '%H:%M').time())

                shift_start_time = current_datetime
                shift_end_time = shift_start_time + timedelta(hours=10)  # Shift ends at 04:00 the next day

                productive_time_including_breaks = (shift_end_time - shift_start_time).total_seconds() / 60 - lunch_duration
                processing_time = (shift_end_time - shift_start_time).total_seconds() / 60 - total_break_time_per_shift - lunch_duration

                schedule.append({
                    'variety': variety,
                    'ShiftStart': shift_start_time,
                    'ShiftEnd': shift_end_time,
                    'ShiftTime': productive_time_including_breaks,  # Productive time includes breaks
                    'BreakTime': total_break_time_per_shift,
                    'LunchBreak': lunch_duration,
                    'ProductiveTime': productive_time_including_breaks,  # Productive Time includes breaks but excludes lunch
                    'ProcessingTime': processing_time  # Processing time excludes both breaks and lunch
                })
                total_time -= productive_time_including_breaks
                current_datetime = shift_end_time

            # After the second shift, reset the date to start Shift A on the next working day at 06:00
            if current_datetime.time() == datetime.strptime('04:00', '%H:%M').time():
                current_datetime = next_working_day(current_datetime)
                current_datetime = datetime.combine(current_datetime, datetime.strptime(daily_start_time, '%H:%M').time())

    return pd.DataFrame(schedule)

# Example usage
start_datetime = datetime(2024, 9, 1, 6, 0)
second_shift_start_time = '18:00'

variety_shift_lengths = {
    'Simcoe': 600,  # 10 hours
    'Citra': 600,   # 10 hours
    'Sabro': 600,   # 10 hours
    'Mosaic': 600,  # 10 hours
    'HBC 1019': 450,  # 7.5 hours
    'Palisade': 450,  # 7.5 hours
    'HBC 522': 450,   # 7.5 hours
    'Talus': 450,     # 7.5 hours
    'HBC 682': 450,   # 7.5 hours
    'HBC 638': 450,   # 7.5 hours
    'HBC 586': 450,   # 7.5 hours
    'Experimental': 450  # 7.5 hours
}

# Assuming 'perrault' is your dataframe
shift_schedule = create_shift_schedule_with_breaks(
    data=perrault,
    start_datetime=start_datetime,
    variety_shift_lengths=variety_shift_lengths,
    daily_start_time='06:00',
    second_shift_start_time=second_shift_start_time  # 6:00 PM
)

# Check if the specific shift on 09/21/2024 at 18:00 is included
shift_on_saturday = shift_schedule[(shift_schedule['ShiftStart'] == '2024-09-21 18:00:00')]

print(shift_on_saturday)



     variety          ShiftStart            ShiftEnd  ShiftTime  BreakTime  \
35  HBC 1019 2024-09-21 18:00:00 2024-09-22 04:00:00      570.0         10   

    LunchBreak  ProductiveTime  ProcessingTime  
35          30           570.0           560.0  
